In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

#  Code to segment the foreground from the background:

In [ ]:
image_path ="/kaggle/input/image-proccising/figimagepro7.jpg"
imageseg = cv2.imread(image_path)

# Specify the coordinates of the rectangle manually
x = 5 # top-left x coordinate
y = 5 # top-left y coordinate
width = 1150 # width of the rectangle
height = 1100  # height of the rectangle
rect = (x, y, width, height)

# Create a mask to indicate the areas of the image that belong to the foreground, background, and potential foreground objects
mask = np.zeros(imageseg.shape[:2], np.uint8)
background_model = np.zeros((1, 65), np.float64)
foreground_model = np.zeros((1, 65), np.float64)

# Apply the GrabCut algorithm to segment the foreground from the background
cv2.grabCut(imageseg, mask, rect, background_model, foreground_model, 5, cv2.GC_INIT_WITH_RECT)

# Create a mask where the probable foreground and certain foreground pixels are marked as 1, and the rest are marked as 0
mask = np.where((mask == 2) | (mask == 0), 0, 1).astype('uint8')

# Apply the mask to the original image to extract the foreground
foreground = imageseg * mask[:, :, np.newaxis]

# Display the original image and the extracted foreground
plt.subplot(121), plt.imshow(cv2.cvtColor(imageseg, cv2.COLOR_BGR2RGB))
plt.title("Original Image"), plt.xticks([]), plt.yticks([])
plt.subplot(122), plt.imshow(cv2.cvtColor(foreground, cv2.COLOR_BGR2RGB))
plt.title("Foreground"), plt.xticks([]), plt.yticks([])
plt.show()

# Change background color into white:

In [ ]:
new_background_color = (255, 255, 255) # (B, G, R)
background_mask = np.all(foreground == 0, axis=2)
foreground[background_mask] = new_background_color
plt.imshow(cv2.cvtColor(foreground, cv2.COLOR_BGR2RGB))
plt.title("Foreground with Changed Background Color")
plt.axis("off")
plt.show() 

# contrast stretching for dark image:

In [ ]:
image_path ="/kaggle/input/image-proccising/figimagepro11.jpg"
imgcon = cv2.imread(image_path)

# Darken the image (adjust the factor as needed)
darkened_img= imgcon
# Convert the darkened image to grayscale
gray_img = cv2.cvtColor(darkened_img, cv2.COLOR_BGR2GRAY)

# Apply contrast stretching to each channel individually
contrast_stretched = np.zeros_like(darkened_img)

for i in range(3):  # Iterate over each color channel
    min_intensity = np.min(darkened_img[:, :, i])
    max_intensity = np.max(darkened_img[:, :, i])
    contrast_stretched[:, :, i] = ((darkened_img[:, :, i] - min_intensity) / (max_intensity - min_intensity)) * 255

# Convert the result back to uint8
contrast_stretched = np.uint8(contrast_stretched)

# Plot the histograms and display the images
plt.figure(figsize=(12, 6))

# Plot the histograms of the darkened image
plt.subplot(2, 2, 1)
plt.hist(darkened_img.flatten(), bins=256, range=[0, 256], color='b', alpha=0.5)
plt.title('Original Image Histogram')
# Display the darkened image
plt.subplot(2, 2, 2)
plt.imshow(cv2.cvtColor(darkened_img, cv2.COLOR_BGR2RGB))
plt.title('Original Image')

# Plot the histograms of the contrast-stretched image
plt.subplot(2, 2, 3)
plt.hist(contrast_stretched.flatten(), bins=256, range=[0, 256], color='r', alpha=0.5)
plt.title('Contrast Stretched Image Histogram')

# Display the contrast-stretched image
plt.subplot(2, 2, 4)
plt.imshow(cv2.cvtColor(contrast_stretched, cv2.COLOR_BGR2RGB))
plt.title('Contrast Stretched Image')

# Show the plots
plt.tight_layout()
plt.show()

#  sharpening filter:

In [ ]:
image_path3 ="/kaggle/input/image-proccising/figimagepro3.jpg"
imgsha = cv2.imread(image_path3)

# Apply the sharpening filter
#This kernel enhances the edges and details in the image, resulting in a sharpened appearance. 
#When -1 is used as the depth value, it indicates that the depth of the output image should be the same as the depth of the input image.
sharpened_image = cv2.filter2D(foreground, -1, kernel = np.array([[-1, -1, -1], [-1, 9, -1], [-1, -1, -1]]))

# Convert BGR to RGB for display
# converts the color space of the foreground image 
sharpened_image = cv2.cvtColor(foreground, cv2.COLOR_BGR2RGB)

# Plotting the original and enhanced images
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(8, 4))
#cv2.cvtColor() is used to convert the color space from BGR to RGB before displaying it.
axes[0].imshow(cv2.cvtColor(contrast_stretched, cv2.COLOR_BGR2RGB))
axes[0].set_title('Original Image')
axes[0].axis('off')
axes[1].imshow(sharpened_image)
axes[1].set_title('sharpened image')
axes[1].axis('off')

plt.tight_layout()
plt.show()

# enhance the image using gaussian:

In [ ]:
from skimage import exposure


# Apply Unsharp Masking to enhance clarity
#The blur helps to smooth out noise and reduce high-frequency details in the image.
#(0,0) kernel 5 id SD
blurred = cv2.GaussianBlur(foreground, (0, 0), 5)

# function is used to combine the original image and the blurred image.

sharpened = cv2.addWeighted(foreground, 1.5, blurred, -0.5, 0)
# Display the original and enhanced images
fig, axs = plt.subplots(1, 2, figsize=(10, 5))
axs[0].imshow(cv2.cvtColor(foreground, cv2.COLOR_BGR2RGB))
axs[0].set_title('Original Image')
axs[0].axis('off')
axs[1].imshow(cv2.cvtColor(sharpened, cv2.COLOR_BGR2RGB))
axs[1].set_title('Enhanced Image')
axs[1].axis('off')
plt.show()

# enhance the image using gamma:

In [ ]:
from skimage import exposure
# Adjust the brightness of the image
#Gamma correction is a non-linear operation that adjusts the brightness of an image by modifying the pixel values.
brightened_image = exposure.adjust_gamma(sharpened, gamma=1.5)

# Plotting the original and brightened images
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(8, 4))
axes[0].imshow(sharpened)
axes[0].set_title('Original Image')
axes[0].axis('off')
axes[1].imshow(brightened_image)
axes[1].set_title('Brightened Image')
axes[1].axis('off')

plt.tight_layout()
plt.show()